In [ ]:
# 导入必要的库
import pandas as pd
import numpy as np

# 读取CSV文件
file_path = 'data/us_asset_returns.csv'
df = pd.read_csv(file_path)

# 删除多余的列（处理CSV文件中可能存在的空列）
df = df.dropna(axis=1, how='all')

# 显示数据基本信息
print("数据基本信息:")
print(df.info())
print("\n前5行数据:")
print(df.head())

In [ ]:
# 处理百分比数据 - 将百分比字符串转换为小数
# 遍历所有列，除了Year列
for col in df.columns:
    if col != 'Year':
        # 移除百分号并转换为浮点数，然后除以100
        df[col] = df[col].str.rstrip('%').astype(float) / 100

# 确保Year列是整数
df['Year'] = df['Year'].astype(int)

# 按年份排序，确保时间序列正确
df = df.sort_values('Year').reset_index(drop=True)

print("\n处理后的数据基本信息:")
print(df.info())
print("\n处理后的前5行数据:")
print(df.head())

In [ ]:
# 使用pandas rolling方法计算十年年化滚动收益率
# 注意：这里计算的是年化收益率，便于不同期限的投资比较

# 为每个资产列计算十年年化滚动收益率
# 自动识别除Year外的所有数值列作为资产列
asset_columns = [col for col in df.columns if col != 'Year' and pd.api.types.is_numeric_dtype(df[col])]

# 如果没有找到数值列，尝试转换百分比字符串列
if not asset_columns:
    # 检查是否有包含百分号的字符串列
    percentage_cols = [col for col in df.columns if col != 'Year' and df[col].dtype == 'object' and df[col].str.contains('%').any()]
    if percentage_cols:
        print(f"检测到百分比字符串列，将自动转换: {percentage_cols}")
        for col in percentage_cols:
            df[col] = df[col].str.rstrip('%').astype(float) / 100
        asset_columns = percentage_cols
    else:
        raise ValueError("未找到有效的资产收益率列")

print(f"自动识别的资产列: {asset_columns}")

for asset in asset_columns:
    rolling_col_name = f'{asset}_十年年化滚动收益率'
    df[rolling_col_name] = (df[asset] + 1).rolling(window=10).apply(
        lambda x: x.prod()**(1/10) - 1, raw=True
    )

In [ ]:
# 显示完整的DataFrame内容
print("\n完整的DataFrame内容（显示前20行和后10行）:")
print("\n前20行数据:")
print(df.head(20))
print("\n后10行数据:")
print(df.tail(10))

In [ ]:
# 显示十年年化滚动收益率的统计信息
print("\n十年年化滚动收益率统计信息:")
rolling_cols = [col for col in df.columns if '十年年化滚动收益率' in col]
for col in rolling_cols:
    valid_data = df[col].dropna()
    if len(valid_data) > 0:
        print(f"\n{col}:")
        print(f"  数据点数: {len(valid_data)}")
        print(f"  平均值: {valid_data.mean():.4f} ({valid_data.mean()*100:.2f}%)")
        print(f"  标准差: {valid_data.std():.4f}")
        print(f"  最小值: {valid_data.min():.4f} ({valid_data.min()*100:.2f}%)")
        print(f"  最大值: {valid_data.max():.4f} ({valid_data.max()*100:.2f}%)")